In [1]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature

In [2]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [3]:
experiment_constraint_data.head(10)

,name,value,priority,constraint_no
0,screen_size,14.4,4,0
1,ram,64,3,0
2,os,11,2,0
3,cpu,i7|ryzen 7,5,0
4,ram,2,2,1
5,price,2699.0,3,1
6,screen_size,15.16,2,1
7,weight,2.06,1,1
8,os,os,1,2
9,camera,True,3,2


In [6]:
problem_list = list()
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    problem: Problem = Problem()
    constraint_list = list()
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint = None
        if item.get('name') in ['brand', 'cpu','gpu','os']:
            constraint: Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['storage', 'ram','price','weight']:
            constraint: Constraint = Constraint(item.get('name'), float(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        elif item.get('name') in ['camera']:
            constraint: Constraint = Constraint(item.get('name'), bool(item.get('value')), item.get('priority'),
                                                attribute_nature.get(item.get('name')))
        constraint_list.append(constraint)
    problem.add_constraint_list(constraint_list)
    problem_list.append(problem)

In [7]:
def get_df_len(df):
    if df is None:
        return 0
    return (df.shape)[0]

for problem in problem_list:
    print(get_df_len(problem.solve()))

137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
137
